<a href="https://colab.research.google.com/github/madhekare/grae/blob/main/daca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRAE: Reinventing Immigration Law Tech

In [1]:
# gdrive specific imports
from google.colab import drive, auth
import gspread
from oauth2client.client import GoogleCredentials

drive.mount('/content/drive', force_remount=False)
auth.authenticate_user()

gc = gspread.authorize(GoogleCredentials.get_application_default())

Mounted at /content/drive


# Pull Data From Google Form

In [2]:
pip install faker

     |████████████████████████████████| 1.2MB 5.5MB/s 


In [3]:
import pandas as pd
import numpy as np
import datetime
import random
from functools import reduce

from faker import Faker
fake = Faker()

# Generate Test Data

In [11]:
def first_row_as_header(df):
  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header #set the header row as the df header
  df = df.reset_index(drop=True)
  return df

def get_gsheet(spreadsheet_info, sheet_name):
  if "docs.google.com" in spreadsheet_info: # is url
    try:
      spreadsheet = gc.open_by_url(spreadsheet_info)
    except:
      raise Exception("Cannot find spreadsheet")
  else: # is name
    spreadsheet = gc.open(spreadsheet_info)

  try:
    worksheet = spreadsheet.worksheet(sheet_name)
  except:
    raise Exception("Cannot find sheet. Sheet names available {}.".format(spreadsheet.worksheets()))

  # get_all_values gives a list of rows.
  rows = worksheet.get_all_values()

  # Convert to a DataFrame and render.
  df = pd.DataFrame.from_records(rows)

  # Get first row as header
  df = first_row_as_header(df)

  return worksheet, df

def randomtimes(start, end, n):
    frmt = '%d-%m-%Y %H:%M:%S'
    stime = datetime.datetime.strptime(start, frmt)
    etime = datetime.datetime.strptime(end, frmt)
    td = etime - stime
    return [random.random() * td + stime for _ in range(n)]

def get_single_date():
    datetime_obj = randomtimes("20-09-2013 13:12:00", "20-09-2013 13:12:00", 1)[0]
    return str(datetime_obj)

def get_random_row():
    result = [get_single_date(),
              fake.name(),
              fake.email(),
              fake.phone_number(),
              fake.street_address(),
              np.random.choice(["Apartment", "Suite", "Floor"]),
              np.random.choice(["Initial Request", "Renewal Request"]),
              np.random.choice(["yes", "no"]),
              np.random.randint(100000, 999999),
              fake.ssn(),
              str(fake.simple_profile()["birthdate"]),
              np.random.choice(["Female", "Male"]),
              fake.city(),
              fake.country(),
              fake.country(),
              fake.country(),
              np.random.choice(["Married", "Widowed", "Single", "Divorced"]),
              np.random.choice(["Hispanic or Latino", "Not Hispanic or latino"]),
              np.random.choice(["White", "Asian", "African American", "American Indian", "Native Hawaiian"]),
              np.random.choice(["yes", "no"]),
              np.random.choice(["true", "false"], p = (.1, .9)),
              np.random.choice(["yes", "no"]),
              np.random.choice(["yes", "no"]),
              np.random.choice(["yes", "no"]),
              np.random.choice(["yes", "no"]),
              np.random.choice(["yes", "no"]),
              np.random.choice(["yes", "no"]),
              np.random.choice(["yes", "no"]),
             ]
    return result

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Name of google sheet
spreadsheet_name = 'DACA Information (Responses)'  # IMPORTANT PARAM
sheet_name = "Form Responses 1"

# Gets sheet
worksheet, df = get_gsheet(spreadsheet_name, sheet_name)
print(len(df), "rows")
df.head()

0 rows


,Timestamp,Full Legal Name,Email,Phone number,US Mailing Address - Street Name and Number,US Mailing Address - type,Requesting:,"Are you NOW or have you EVER been in removal proceedings, or do you have a removal order issued in any other context (for example, at the border or within the United States by an immigration agent)?",Alien Registration Number,US social security number,Date of Birth,Gender,City/town/village of Birth,Country of Birth,Current Country of Residence,Country of Citizenship or Nationality,Martial Status,Ethnicity,Race,"I have been continuously residing in the U.S. since at least June 15, 2007, up to the present time.","Have you EVER been arrested for, charged with, or convicted of a felony or misdemeanor, including incidents handled in juvenile court, in the United States? Do not include minor traffic violations unless they were alcohol- or drug-related",Were you under 16 years old when entering the United States of America?,"Were you under 31 years old as of June 15, 2012?","Have you continuously resided in the U.S. from June 15, 2007, to the present?","Did you enter the U.S. without inspection before June 15, 2012, or, if you had legal immigration status, it had expired as of June 15, 2012?","Were you physically present in the U.S. on June 15, 2012 (you must also be physically present in the U.S. when you apply for DACA)?","Are you currently in school, have you graduated from high school, obtained your GED, or have you been honorably discharged from the U.S. armed forces or Coast Guard?",Do you pose a threat to national security or public safety?


In [14]:
cols = list(df.columns)
cols

['Timestamp',
 'Full Legal Name',
 'Email',
 'Phone number',
 'US Mailing Address - Street Name and Number',
 'US Mailing Address - type ',
 'Requesting:',
 'Are you NOW or have you EVER been in removal proceedings, or do you have a removal order issued in any other context (for example, at the border or within the United States by an immigration agent)?',
 'Alien Registration Number',
 'US social security number',
 'Date of Birth',
 'Gender',
 'City/town/village of Birth',
 'Country of Birth',
 'Current Country of Residence',
 'Country of Citizenship or Nationality',
 'Martial Status',
 'Ethnicity',
 'Race',
 'I have been continuously residing in the U.S. since at least June 15, 2007, up to the present time.',
 'Have you EVER been arrested for, charged with, or convicted of a felony or misdemeanor, including incidents handled in juvenile court, in the United States? Do not include minor traffic violations unless they were alcohol- or drug-related',
 'Were you under 16 years old when e

In [15]:
num_rows = 100
for i in range(num_rows):
    df = df.append(dict(zip(cols, get_random_row())), ignore_index=True)

In [16]:
df

,Timestamp,Full Legal Name,Email,Phone number,US Mailing Address - Street Name and Number,US Mailing Address - type,Requesting:,"Are you NOW or have you EVER been in removal proceedings, or do you have a removal order issued in any other context (for example, at the border or within the United States by an immigration agent)?",Alien Registration Number,US social security number,Date of Birth,Gender,City/town/village of Birth,Country of Birth,Current Country of Residence,Country of Citizenship or Nationality,Martial Status,Ethnicity,Race,"I have been continuously residing in the U.S. since at least June 15, 2007, up to the present time.","Have you EVER been arrested for, charged with, or convicted of a felony or misdemeanor, including incidents handled in juvenile court, in the United States? Do not include minor traffic violations unless they were alcohol- or drug-related",Were you under 16 years old when entering the United States of America?,"Were you under 31 years old as of June 15, 2012?","Have you continuously resided in the U.S. from June 15, 2007, to the present?","Did you enter the U.S. without inspection before June 15, 2012, or, if you had legal immigration status, it had expired as of June 15, 2012?","Were you physically present in the U.S. on June 15, 2012 (you must also be physically present in the U.S. when you apply for DACA)?","Are you currently in school, have you graduated from high school, obtained your GED, or have you been honorably discharged from the U.S. armed forces or Coast Guard?",Do you pose a threat to national security or public safety?
0,2013-09-20 13:12:00,Mark Daugherty,veronicajordan@phelps-pratt.com,554.755.9054,0419 Michael Fort,Floor,Initial Request,no,425897,025-12-7237,2009-10-14,Male,Port Michael,French Polynesia,Ethiopia,Samoa,Widowed,Not Hispanic or latino,African American,no,false,yes,no,no,no,yes,no,no
1,2013-09-20 13:12:00,Michele Miller,jesse98@yahoo.com,+1-259-758-5519x6136,6877 Allen Extension,Suite,Initial Request,no,601058,070-27-8948,1943-12-22,Female,Kelleyport,Korea,Cook Islands,Vanuatu,Widowed,Not Hispanic or latino,White,no,false,no,yes,yes,no,no,no,no
2,2013-09-20 13:12:00,Edward Willis,richard21@gonzalez-spears.info,(042)464-6480,948 Jessica Passage Suite 510,Apartment,Initial Request,yes,282230,214-22-2082,1975-04-02,Female,South Shannonfort,Dominican Republic,South Africa,Aruba,Widowed,Not Hispanic or latino,African American,yes,true,no,no,no,no,yes,no,no
3,2013-09-20 13:12:00,Brittany Sexton,christinegraham@villegas.biz,323-592-5777,4875 Rogers Spring Suite 734,Suite,Renewal Request,no,929783,899-42-8091,1950-12-06,Male,North Joeton,Grenada,Switzerland,Liechtenstein,Divorced,Hispanic or Latino,Native Hawaiian,yes,false,no,no,yes,yes,no,yes,no
4,2013-09-20 13:12:00,Adrian Shepherd,snowmark@booth-robinson.com,001-071-608-3133x82795,892 Romero Throughway,Suite,Renewal Request,no,807800,183-07-2245,1962-11-11,Female,Maybury,Solomon Islands,Guam,Palestinian Territory,Widowed,Hispanic or Latino,Asian,yes,false,yes,no,no,yes,yes,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2013-09-20 13:12:00,Chad Sullivan,anthonywilliams@williams.org,327.713.1854x0643,855 Fuentes Flats,Suite,Initial Request,no,194653,838-02-9837,1994-12-21,Female,North Danaside,Algeria,Guernsey,Sao Tome and Principe,Divorced,Hispanic or Latino,White,no,false,yes,no,yes,no,no,no,yes
96,2013-09-20 13:12:00,Margaret Baird,richardburns@powell-fisher.com,001-992-061-2688,9592 Sheila Knoll,Suite,Renewal Request,yes,943625,857-97-0256,1954-04-20,Female,West John,Liechtenstein,Spain,Bahrain,Single,Hispanic or Latino,Native Hawaiian,no,false,yes,yes,no,yes,yes,no,yes
97,2013-09-20 13:12:00,Tyler Gonzalez,hschmidt@gmail.com,913-306-7345x72216,77091 David Turnpike,Apartment,Renewal Request,no,444282,296-99-7095,1922-02-08,Male,Jenkinstown,Central African Republic,Nigeria,San Marino,Divorced,Not Hispanic or latino,White,yes,false,yes,no,ye

In [ ]:
# df["Flag"] = [np.random.choice([True, False]) for i in range(len(df))]

# Evaluate DACA Applicatant

1. added questions to google form, and edit get_random_row function to include random data for these new questions -- done!!✅
2. write a function, given an applicant (a row of data) whether or not they are rejected or accepted based on https://www.chavezlawgroup.com/daca-immigration/ -- done!!✅

In [30]:
def evaluate_form(row):
    condition = row["Have you EVER been arrested for, charged with, or convicted of a felony or misdemeanor, including incidents handled in juvenile court, in the United States? Do not include minor traffic violations unless they were alcohol- or drug-related"] == "false"
    #return (datetime.today() - row["DOB"]) > datetime.datetime(10 years)
    if not condition:
      return "FLAGGED"
    for i in range(21,28):
      if row[i] != 'yes': 
        return "FLAGGED" 
    return "good"

In [31]:
evaluate_form(df.iloc[1])

'FLAGGED'

# Sentiment Analysis for Petition

In [32]:
pip install transformers

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 3.3MB 19.7MB/s 
     |████████████████████████████████| 901kB 35.6MB/s 


In [33]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 9.6MB/s 
     |████████████████████████████████| 901kB 37.3MB/s 
     |████████████████████████████████| 3.3MB 49.2MB/s 


In [2]:
from transformers import pipeline

In [4]:
sentiment_analysis = pipeline("sentiment-analysis")

In [17]:
#Work visa petition exapmles: https://www.usavisanow.com/h-1b-visa/employeremployee/sample-letter-to-uscis-in-house-consulting-company/
pos_text = "About the Petitioner – Acme, Inc. Headquartered in New York, Acme is a nine-year old global provider of high quality and cost-effective Information Technology Services.  We offer a range of expertise aimed at helping customers reengineer/reinvent their businesses to compete successfully in an ever changing marketplace.  We boast of one of the highest customer retention rates in the industry.Acme’s I-129-H Petition on behalf of Mr. Singh meets all the required criteria that a Petitioner must establish for the existence of an employer-employee relationship and that this relationship will continue to exist with the Beneficiary throughout the duration of the requested H-1B validity period."

neg_text = "Acme does not maintain much supervision"

In [18]:
result = sentiment_analysis(pos_text)[0]
print("Label:", result['label'])
print("Confidence Score:", result['score'])
print()
result = sentiment_analysis(neg_text)[0]
print("Label:", result['label'])
print("Confidence Score:", result['score'])

Label: POSITIVE
Confidence Score: 0.99933260679245

Label: NEGATIVE
Confidence Score: 0.9997295141220093
